https://www.kaggle.com/its7171/cv-strategy

In [1]:
import pandas as pd
import random
import gc

random.seed(1)

import sys
sys.path.append('../src')
import const

In [2]:
train = pd.read_csv(const.INPUT_DATA_DIR / 'train.csv',dtype=const.DTYPE)

In [3]:
valid_split1 = train.groupby('user_id').tail(5)
train_split1 = train[~train.row_id.isin(valid_split1.row_id)]
valid_split1 = valid_split1[valid_split1.content_type_id == 0]
train_split1 = train_split1[train_split1.content_type_id == 0]
print(f'{train_split1.answered_correctly.mean():.3f} {valid_split1.answered_correctly.mean():.3f}')

0.660 0.541


In [4]:
max_timestamp_u = train[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u.columns = ['user_id', 'max_time_stamp']
MAX_TIME_STAMP = max_timestamp_u.max_time_stamp.max()

In [5]:
def rand_time(max_time_stamp):
    interval = MAX_TIME_STAMP - max_time_stamp
    rand_time_stamp = random.randint(0,interval)
    return rand_time_stamp

max_timestamp_u['rand_time_stamp'] = max_timestamp_u.max_time_stamp.apply(rand_time)
train = train.merge(max_timestamp_u, on='user_id', how='left')
train['viretual_time_stamp'] = train.timestamp + train['rand_time_stamp']

In [6]:
kaggle_env = False
if kaggle_env:
    # Full dataframe can not be sorted on kaggle kernel due to lack of memory.
    train = train[:10_000_000]
train = train.sort_values(['viretual_time_stamp', 'row_id']).reset_index(drop=True)

In [7]:
if kaggle_env:
    val_size = 250_000
else:
    val_size = 2_500_000

for cv in range(5):
    valid = train[-val_size:]
    train = train[:-val_size]
    # check new users and new contents
    new_users = len(valid[~valid.user_id.isin(train.user_id)].user_id.unique())
    valid_question = valid[valid.content_type_id == 0]
    train_question = train[train.content_type_id == 0]
    new_contents = len(valid_question[~valid_question.content_id.isin(train_question.content_id)].content_id.unique())    
#     print(f'cv{cv} {train_question.answered_correctly.mean():.3f} {valid_question.answered_correctly.mean():.3f} {new_users} {new_contents}')
    valid.to_pickle(f'../folds/cv{cv+1}_valid.pickle')
    train.to_pickle(f'../folds/cv{cv+1}_train.pickle')